In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import local_binary_pattern

In [4]:
# ==== PARAMETERS ====
INPUT_DIR = "C:/Users/adity/Documents/Pyhton_practice/streptomyces_model_2/dataset_raw/Streptomyces"  # Raw dataset folder with subfolders for each class
OUTPUT_DIR = "C:/Users/adity/Documents/Pyhton_practice/streptomyces_model_2/Streptomyces/dataset_processed"  # Where processed images will go
IMG_SIZE = (512, 512)
LBP_RADIUS = 3
LBP_POINTS = 8 * LBP_RADIUS

In [5]:
print("🔍 Contents of dataset_raw:", os.listdir(INPUT_DIR))

🔍 Contents of dataset_raw: ['DSM_41600_image3.jpeg', 'DSM_41600_image4.jpeg', 'img_001.jpg', 'img_002.jpg', 'img_003.jpg', 'img_004.jpg', 'img_005.jpg', 'img_006.jpg', 'img_007.jpg', 'img_008.jpg', 'img_009.jpg', 'img_010.jpg', 'img_011.jpg', 'img_012.jpg', 'img_013.jpg', 'img_014.jpg', 'img_015.jpg', 'img_016.jpg', 'img_017.jpg', 'img_018.jpg', 'img_019.jpg', 'img_020.jpg', 'img_021.jpg', 'img_022.jpg', 'img_023.jpg', 'img_024.jpg', 'img_025.jpg', 'img_026.jpg', 'img_027.jpg', 'img_028.jpg', 'img_029.jpg', 'img_030.jpg', 'img_031.jpg', 'img_032.jpg', 'img_033.jpg', 'img_034.jpg', 'img_035.jpg', 'img_036.jpg', 'img_037.jpg', 'img_038.jpg', 'img_039.jpg', 'img_040.jpg', 'img_041.jpg', 'img_042.png', 'img_043.jpg', 'img_044.jpg', 'img_045.jpg', 'img_046.jpg', 'img_047.jpg', 'img_048.jpg', 'img_049.jpg', 'img_050.jpg', 'img_051.jpg', 'img_052.jpg', 'img_053.jpg', 'img_054.jpg', 'img_055.jpg', 'img_056.jpg', 'img_057.jpg', 'img_058.jpg', 'img_059.jpg', 'img_060.jpg', 'img_061.jpg', 'img_06

In [6]:
# ==== CREATE OUTPUT DIR STRUCTURE ====
classes = [cls for cls in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, cls))]
for cls in classes:
    os.makedirs(os.path.join(OUTPUT_DIR, cls), exist_ok=True)

In [7]:
# ==== LBP Texture Function ====
def generate_lbp (gray_img, radius = LBP_RADIUS, points = LBP_POINTS):
    lbp = local_binary_pattern(gray_img, points, radius, method='uniform')
    lbp = ((lbp - lbp.min())/ (lbp.max() - lbp.min()) * 255).astype(np.unit8)
    return lbp

In [8]:
# ==== CLAHE on Saturation Channel ====
def enhance_saturation_clahe(bgr_img):
    # Convert image from BGR to HSV color space
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    # Create CLAHE object for local contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # Apply CLAHE to the saturation channel
    hsv[..., 1] = clahe.apply(hsv[..., 1])
    # Convert back to BGR
    enhanced = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return enhanced

In [56]:
# ==== HED Edge Map using OpenCV DNN ====
# ==== HED Edge Map using OpenCV DNN ====
def generate_hed_edges(img):
    proto = "C:/Users/adity/Documents/Pyhton_practice/hed_model/deploy.prototxt"  # Path to model architecture
    weights = "C:/Users/adity/Documents/Pyhton_practice/hed_model/hed_pretrained_bsds.caffemodel"  # Path to pretrained weights

    if not os.path.exists(proto) or not os.path.exists(weights):
        print("❌ HED model files not found.")
        return np.zeros(IMG_SIZE, dtype=np.uint8)  # Return blank if model is missing

    net = cv2.dnn.readNetFromCaffe(proto, weights)

    blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(IMG_SIZE[0], IMG_SIZE[1]),
                                 mean=(104.00698793, 116.66876762, 122.67891434), swapRB=False, crop=False)
    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], IMG_SIZE)
    hed = (255 * hed).astype(np.uint8)
    return hed
    

In [9]:
for cls in tqdm(classes, desc="Processing classes"):
    class_path = os.path.join(INPUT_DIR, cls)
    save_path = os.path.join(OUTPUT_DIR, cls)

    for fname in os.listdir(class_path):
        print(f"🔍 Found file: {fname} in class: {cls}")
        if not fname.lower().endswith(('.JPG', '.JPEG', '.PNG')):
            continue

        img_path = os.path.join(class_path, fname)
        img = cv2.imread(img_path)
        if img is None:
           print(f"⚠️ Could not read image: {img_path}")
           continue
        else:
            print(f"📸 Image loaded: {img.shape}")




        img = cv2.resize(img, IMG_SIZE)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = generate_lbp(gray)
        clahe_img = enhance_saturation_clahe(img)
        hsv = cv2.cvtColor(clahe_img, cv2.COLOR_BGR2HSV)
        sat_clahe = hsv[..., 1]

       # Minimal test: LBP + CLAHE (duplicate for 3rd channel)
        composite = cv2.merge([lbp, sat_clahe, sat_clahe])

        # Debug info
        print(f"🧪 Checking composite types: LBP({lbp.dtype}), CLAHE({sat_clahe.dtype})")
        print(f"🧪 Composite shape: {composite.shape}, dtype: {composite.dtype}, max: {composite.max()}")

        composite = np.clip(composite, 0, 255).astype(np.uint8)

        save_name = os.path.splitext(fname)[0] + '_prep.png'
        save_file = os.path.join(save_path, save_name)
        success = cv2.imwrite(save_file, composite)

        if not success:
            print(f"❌ Failed to save image: {save_file}")
        else:
            print(f"✅ Saved: {save_file}")

print("✅ LBP + CLAHE-only images saved in:", OUTPUT_DIR)



Processing classes: 0it [00:00, ?it/s]

✅ LBP + CLAHE-only images saved in: C:/Users/adity/Documents/Pyhton_practice/streptomyces_model_2/Streptomyces/dataset_processed


In [10]:
import os
import cv2
from matplotlib import pyplot as plt

folder = 'C:/Users/adity/Documents/Pyhton_practice/streptomyces_model_2/dataset_processed/Streptomyces'  # or micromonospora
files = [f for f in os.listdir(folder) if f.endswith('_prep.png')]

for f in files[:5]:
    path = os.path.join(folder, f)
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f)
    plt.axis('off')
    plt.show()